In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import warnings

from utils2_check import *
from stat_utils_check import *

pd.set_option('future.no_silent_downcasting', True)
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore')


In [2]:
path = "C:/Users/82104/Downloads/Insurance_fraud/"
df = pd.read_csv(path + "fraud_oracle.csv")

> # Tip #1 레퍼런스를 찾아라

- https://www.quora.com/Can-a-person-give-a-wrong-address-to-obtain-a-cheep-car-insurances
- https://www.allstate.com/resources/car-insurance/types-of-car-insurance-fraud
- https://www.aura.com/learn/change-of-address-scam

# 주소를 이전한 경우, 보험 사기일 가능성이 높을 것이다.
- (기준에 대한 구체화) 주소를 언제 이전? 최근? 옛날에? 최근이나 옛날의 기준은?
    - -> 주소 이전 시기에 따라 보험 사기의 패턴이 있을 것이다
- (다른 변수와의 관계 구체화) 주소를 이사로 인해 이전한 경우와 어떻게 구분하지?
    - -> 결혼 여부, 사고 여부 등 주소 이전 사유와 관련이 있을 것이다.
- (비즈니스 근거) 
    - 위치에 따라 보험료가 달라지기에, 거짓된 주소를 기입
    - 또는, 보험금 수령 주소를 변경하여 대신 수령 등
- (결론 및 활용) 비용/자원 여부와 비즈니스 타당성 여부를 함께 고려
    - 보험 사기 탐지에 활용

In [3]:
df.head(2)

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,FraudFound_P,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,21,Policy Holder,Sport - Liability,Sport,more than 69000,0,1,12,300,1,more than 30,more than 30,none,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,34,Policy Holder,Sport - Collision,Sport,more than 69000,0,2,15,400,4,more than 30,more than 30,none,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision


In [4]:
plot_count_plot(df, "AddressChange_Claim", "FraudFound_P")

FraudFound_P


각 상품별로 데이터 구분

In [5]:
collision = df.loc[df['BasePolicy']=="Collision"]
liability = df.loc[df['BasePolicy']=="Liability"]
all_p = df.loc[df['BasePolicy']=="All Perils"]

In [6]:
o, c = statistical_tests_step1(all_p, "FraudFound_P", ['AddressChange_Claim'])
o, c = statistical_tests_step1(collision, "FraudFound_P", ['AddressChange_Claim'])

Relevant : AddressChange_Claim
Relevant : AddressChange_Claim


---

In [8]:
# plot_count_plot(collision, 'AddressChange_Claim', 'FraudFound_P')
plot_two_categorical(collision, 'AddressChange_Claim', 'FraudFound_P', 'stacked_barplot')

In [9]:
plot_count_plot(all_p, 'AddressChange_Claim', 'FraudFound_P')
plot_two_categorical(all_p, 'AddressChange_Claim', 'FraudFound_P', 'stacked_barplot')

FraudFound_P


> # Tip #4 : 결과는 숫자로 표현하라

In [10]:
# help(pd.crosstab)
pd.crosstab(all_p['AddressChange_Claim'], all_p['FraudFound_P'], normalize='columns')

FraudFound_P,0,1
AddressChange_Claim,,
1 year,0.010008,0.017699
2 to 3 years,0.013260,0.048673
4 to 8 years,0.040781,0.030973
no change,0.935952,0.896018
under 6 months,0.000000,0.006637


In [11]:
all_p.FraudFound_P.value_counts()

FraudFound_P
0    3997
1     452
Name: count, dtype: int64

In [12]:
pd.crosstab(collision['AddressChange_Claim'], collision['FraudFound_P'], normalize='columns')

FraudFound_P,0,1
AddressChange_Claim,,
1 year,0.011037,0.006897
2 to 3 years,0.014293,0.064368
4 to 8 years,0.041614,0.041379
no change,0.933056,0.887356


- 보험 사기가 아닌 경우, 주소 이전이 없는 경우가 93%다.
- 반면, 보험 사기인 경우 주소 이전이 없는 경우가 89%로 4% 낮다.
- 대신, 2~3년 이내 주소 이전한 경우가 4% 높다.

---

> # Tip #3 : 이해하기 쉬운 형태로 변형하라

- 특정 카테고리에 data point가 적은 경우가 있다. 또한, 이해하기 쉬운 형태로 더 간단하게 변형
    - 변경 O VS 변경 X
    - 최근 3년 이내 변경 여부 VS 변경 X VS 오래전에 변경

In [13]:
output, relevant_col = statistical_tests_step1(all_p, 'FraudFound_P', ['AddressChange_Claim'])
# print(output)

Relevant : AddressChange_Claim


In [14]:
output, relevant_col = statistical_tests_step1(collision, 'FraudFound_P', ['AddressChange_Claim'])
# print(output)

Relevant : AddressChange_Claim


In [15]:
address_change_mapping = {
    '1 year' : "recent",
    'no change' : "no_change",
    '4 to 8 years' : "old", 
    '2 to 3 years' : "recent",
    'under 6 months' : "recent"
}

collision['AddressChange_Claim_2'] = collision['AddressChange_Claim'].map(address_change_mapping)
all_p['AddressChange_Claim_2'] = all_p['AddressChange_Claim'].map(address_change_mapping)

In [17]:
collision['AddressChange_Claim_2'].unique()

array(['no_change', 'old', 'recent'], dtype=object)

In [18]:
collision['AddressChange_Claim_2'].value_counts()

AddressChange_Claim_2
no_change    5543
old           248
recent        171
Name: count, dtype: int64

In [19]:
output, relevant_col = statistical_tests_step1(all_p, 'FraudFound_P', ['AddressChange_Claim_2'])
print(output)

Relevant : AddressChange_Claim_2
{'AddressChange_Claim_2': {'chi2': 37.15290686307279, 'p': 8.557536917030331e-09, 'dof': 2, 'expected': array([[3724.78354686,  421.21645314],
       [ 159.01753203,   17.98246797],
       [ 113.19892111,   12.80107889]]), 'contingency_table': FraudFound_P              0    1
AddressChange_Claim_2           
no_change              3741  405
old                     163   14
recent                   93   33}}


In [20]:
output, relevant_col = statistical_tests_step1(collision, 'FraudFound_P', ['AddressChange_Claim_2'])
print(output)

Relevant : AddressChange_Claim_2
{'AddressChange_Claim_2': {'chi2': 30.572072027143463, 'p': 2.29804701287303e-07, 'dof': 2, 'expected': array([[5138.57111707,  404.42888293],
       [ 229.90540087,   18.09459913],
       [ 158.52348205,   12.47651795]]), 'contingency_table': FraudFound_P              0    1
AddressChange_Claim_2           
no_change              5157  386
old                     230   18
recent                  140   31}}


In [21]:
# 변수 단순화
address_change_mapping = {
    '1 year' : "change",
    'no change' : "no_change",
    '4 to 8 years' : "change", 
    '2 to 3 years' : "change",
    'under 6 months' : "change"
}

collision['AddressChange_Claim_3'] = collision['AddressChange_Claim'].map(address_change_mapping)
all_p['AddressChange_Claim_3'] = all_p['AddressChange_Claim'].map(address_change_mapping)

In [22]:
collision.AddressChange_Claim_3.unique()

array(['no_change', 'change'], dtype=object)

In [23]:
output, relevant_col = statistical_tests_step1(all_p, 'FraudFound_P', ['AddressChange_Claim_3'])
print(output)

Relevant : AddressChange_Claim_3
{'AddressChange_Claim_3': {'chi2': 9.584110044716049, 'p': 0.001962685501944352, 'dof': 1, 'expected': array([[ 272.21645314,   30.78354686],
       [3724.78354686,  421.21645314]]), 'contingency_table': FraudFound_P              0    1
AddressChange_Claim_3           
change                  256   47
no_change              3741  405}}


In [24]:
output, relevant_col = statistical_tests_step1(collision, 'FraudFound_P', ['AddressChange_Claim_3'])
print(output)

Relevant : AddressChange_Claim_3
{'AddressChange_Claim_3': {'chi2': 12.199576497395398, 'p': 0.0004780036610444878, 'dof': 1, 'expected': array([[ 388.42888293,   30.57111707],
       [5138.57111707,  404.42888293]]), 'contingency_table': FraudFound_P              0    1
AddressChange_Claim_3           
change                  370   49
no_change              5157  386}}


주소 이전을 한 경우, 얼마나 달라질까?

In [25]:
plot_two_categorical(all_p, 'FraudFound_P', 'AddressChange_Claim_3', 'heatmap')

In [26]:
# 주소 이전한 사람이 안한사람보다 5%정도 높다
# 관련 있다.
pd.crosstab(collision['FraudFound_P'], collision['AddressChange_Claim_3'], normalize='columns')

AddressChange_Claim_3,change,no_change
FraudFound_P,,
0,0.883055,0.930363
1,0.116945,0.069637


In [27]:
pd.crosstab(all_p['FraudFound_P'], all_p['AddressChange_Claim_3'], normalize='columns')

AddressChange_Claim_3,change,no_change
FraudFound_P,,
0,0.844884,0.902315
1,0.155116,0.097685


주소를 이전한 경우 15%가 보험 사기었다.
주소를 이전하지 않은 경우는 9%가 보험 사기.
- 즉, 주소 이전을 통한 보험 사기를 걸러낼 수 있는 수단인 것이다
- 하지만 주소 이전을 했다고 해서, 무조건 보험사기는 아니기 때문에, 다른 변수와의 관계를 통해 패턴을 더 알아내고자 한다

---

> # 원칙 1 : 다양한 변수간의 관계를 살펴볼 것

어떤 특징을 갖는 사람들이 주소 이전을 통한 보험 사기를 칠까?

주소 이전을 통한 보험 사기를 구분할 수 있는 다른 변수들은 없을까?

In [28]:
time_vars = ["Month", "WeekOfMonth", "DayOfWeek", "DayOfWeekClaimed", 'MonthClaimed', 'WeekOfMonthClaimed']
vehicle_vars = ["Make", "VehiclePrice", "VehicleCategory", "AgeOfVehicle"]
personal_vars = ["Sex", 'MaritalStatus', "Age", 'DriverRating', 'AgeOfPolicyHolder', 'NumberOfCars', 'PastNumberOfClaims']
policy_vars = ["PolicyType", 'Deductible', 'AgentType', "NumberOfSuppliments", "BasePolicy"]
accident_vars = ['Days_Policy_Accident', 'Days_Policy_Claim', 'PoliceReportFiled', 'WitnessPresent', 'AddressChange_Claim']

In [29]:
# 주소변경 하지 않은 사람들
all_nochange = all_p.loc[all_p['AddressChange_Claim_3']=='no_change']

o, c = statistical_tests_step1(all_nochange, 'FraudFound_P', policy_vars + vehicle_vars + personal_vars + accident_vars)

Relevant : Make
Relevant : VehiclePrice
Relevant : AgeOfVehicle
Relevant : Age
Relevant : AgeOfPolicyHolder
Relevant : PoliceReportFiled


In [30]:
# 주소변경한 사람들
# 궁극적으로 주소변경한 사람들과 관련있는 것을 찾고싶다.
# 여기서 통계적 의미가 있고 위의 주소 변경 안한 사람들에서 의미가 없는
# 변수를 찾는다.
all_change = all_p.loc[all_p['AddressChange_Claim_3']=='change']

o, c = statistical_tests_step1(all_change, 'FraudFound_P', policy_vars + vehicle_vars + personal_vars + accident_vars)

Relevant : Deductible
Relevant : AgeOfVehicle
Relevant : Age
Relevant : AgeOfPolicyHolder
Relevant : NumberOfCars
Relevant : AddressChange_Claim


두 번의 테스트 중, address_change 데이터에서만 유의미하게 나온 column들을 선택해보자.
- 주소 변경이 없는 곳에서는 의미가 없지만, 주소 변경을 한 사람들을 대상으로 했을 때에만 유의미하게 나오는 컬럼들이기 때문에, 주소 변경을 한 피보험자들에 국한되어 추가적인 패턴을 찾을 수 있기 때문이다.

-> `Deductible` & `NumberOfCars`

In [31]:
# 마찬가지.
collision_nochange = collision.loc[collision['AddressChange_Claim_3']=='no_change']

o, c = statistical_tests_step1(collision_nochange, 'FraudFound_P', policy_vars + vehicle_vars + personal_vars + accident_vars)

Relevant : PolicyType
Relevant : VehiclePrice
Relevant : VehicleCategory
Relevant : Sex


In [32]:
collision_change = collision.loc[collision['AddressChange_Claim_3']=='change']

o, c = statistical_tests_step1(collision_change, 'FraudFound_P', policy_vars + vehicle_vars + personal_vars + accident_vars)

Relevant : Deductible
Relevant : VehiclePrice
Relevant : AgeOfVehicle
Relevant : NumberOfCars
Relevant : Days_Policy_Accident
Relevant : AddressChange_Claim


-> `Deductible` & `NumberOfCars` & `Days_Policy_Accident`

두개 다 Deductible, NumberOfCars가 있기때문에
이 두개를 메인으로 분석

In [33]:
plot_two_categorical(collision_change,'FraudFound_P', 'NumberOfCars', 'stacked_barplot')
# 주소를 이전한 대상 중, 보험사기인 사고의 경우 여러대가 아닌 한 대를 갖고 있는 피보험사들의 비율이 더 높다

In [34]:
plot_two_categorical(all_change,'FraudFound_P', 'NumberOfCars', 'stacked_barplot')
# 주소를 이전한 대상 중, 보험사기인 사고의 경우 여러대가 아닌 한 대를 갖고 있는 피보험사들의 비율이 더 높다

In [35]:
# 자기부담금 = Deductible
# 처음 보험 상품 가입할 때 설정함. 높아질수록 사고가 날때 그만큼 본인이 내야함
# 하지만 많아지는만큼 커버해주는 보험금도 커짐.
# 비싼차량, 더 안전하게 많은 커버리지를 가져가고싶을때 높게 설정하여 가입

plot_two_categorical(collision_change,'FraudFound_P', 'Deductible', 'stacked_barplot')
# 주소를 이전한 대상 중, 보험사기인 사고의 경우 여러대가 아닌 한 대를 갖고 있는 피보험사들의 비율이 더 높다

In [36]:
plot_two_categorical(all_change,'FraudFound_P', 'Deductible', 'stacked_barplot')
# 주소를 이전한 대상 중, 보험사기인 사고의 경우 여러대가 아닌 한 대를 갖고 있는 피보험사들의 비율이 더 높다

> # Tip #4 : 결과는 숫자로 표현하라

In [38]:
tmp = all_p.loc[all_p['AddressChange_Claim_3']=='no_change']

statistical_tests_step1(tmp, 'FraudFound_P', ['NumberOfCars'])

pd.crosstab(tmp['NumberOfCars'], tmp['FraudFound_P'], normalize='columns')

FraudFound_P,0,1
NumberOfCars,,
1 vehicle,0.975675,0.967901
3 to 4,0.022454,0.029630
5 to 8,0.001604,0.002469
more than 8,0.000267,0.000000


최근에 주소를 변경하지 않은 경우, 
1. 기존의 패턴과 유사하다
2. 통계적으로 유의미하지 않다
- -> 사기 여부를 판단하는데에 도움이 되지 않는다.

In [39]:
# 주소 이전 관련 데이터 중에서 차 한개일때 30퍼가 보험사기와 관련됨
tmp = all_p.loc[all_p['AddressChange_Claim_3']=='change']

statistical_tests_step1(tmp, 'FraudFound_P', ['NumberOfCars'])

pd.crosstab(tmp['FraudFound_P'], tmp['NumberOfCars'], normalize='columns')

Relevant : NumberOfCars


NumberOfCars,1 vehicle,2 vehicles,3 to 4
FraudFound_P,,,
0,0.698925,0.906404,1.0
1,0.301075,0.093596,0.0


In [40]:
pd.crosstab(tmp['NumberOfCars'], tmp['FraudFound_P'])

FraudFound_P,0,1
NumberOfCars,,
1 vehicle,65,28
2 vehicles,184,19
3 to 4,7,0


### 주소를 변경한 이력이 있는 경우 차량의 대수가 보험 사기를 판단하는데 유의미해진다.

- 주소 이전을 했다 : 
    - 차량을 1대 갖고 있다 -> 보험 사기일 확률 28/(28+65) = 30%
    - 차량을 2대 갖고 있다 -> 보험 사기일 확률 19/(19+184) = 9%
- 따라서 주소 이전을 한 사람들을 대상으로 보험 사기를 방지하는 절차 마련 필요 (이때, 차량의 대수와 관련이 있는지 여부도 함께 조사 필요)

In [41]:
tmp = collision.loc[collision['AddressChange_Claim_3']=='change']

statistical_tests_step1(tmp, 'FraudFound_P', ['NumberOfCars'])

pd.crosstab(tmp['FraudFound_P'], tmp['NumberOfCars'], normalize='columns')

Relevant : NumberOfCars


NumberOfCars,1 vehicle,2 vehicles,3 to 4
FraudFound_P,,,
0,0.813433,0.920578,0.75
1,0.186567,0.079422,0.25


In [42]:
pd.crosstab(tmp['FraudFound_P'], tmp['NumberOfCars'])

NumberOfCars,1 vehicle,2 vehicles,3 to 4
FraudFound_P,,,
0,109,255,6
1,25,22,2


- 주소 이전을 했다 : 
    - 차량을 1대 갖고 있다 -> 보험 사기일 확률 25/(25+109) = 18%
    - 차량을 2대 갖고 있다 -> 보험 사기일 확률 22/(22+255) = 7%
- 상대적으로 패턴이 약하기는하지만, 주소 이전한 피보험자들 중 1대의 차량을 갖고 있는 경우 보험사기에 노출될 가능성이 상대적으로 더 높다

자기부담금

---
Deductible도 살펴보자

In [43]:
tmp = all_p.loc[all_p['AddressChange_Claim_3']=='no_change']

statistical_tests_step1(tmp, 'FraudFound_P', ['Deductible'])

pd.crosstab(tmp['Deductible'], tmp['FraudFound_P'], normalize='columns')

# plot_count_plot(tmp, 'Deductible', 'FraudFound_P')

FraudFound_P,0,1
Deductible,,
400,0.979417,0.977778
500,0.000802,0.000000
700,0.019781,0.022222


In [44]:
tmp = all_p.loc[all_p['AddressChange_Claim_3']=='change']

statistical_tests_step1(tmp, 'FraudFound_P', ['Deductible'])

pd.crosstab(tmp['Deductible'], tmp['FraudFound_P'], normalize='columns')
# plot_count_plot(tmp, 'Deductible', 'FraudFound_P')

Relevant : Deductible


FraudFound_P,0,1
Deductible,,
300,0.003906,0.021277
400,0.796875,0.510638
500,0.171875,0.446809
700,0.027344,0.021277


In [45]:
pd.crosstab(tmp['Deductible'], tmp['FraudFound_P'], normalize='columns')

FraudFound_P,0,1
Deductible,,
300,0.003906,0.021277
400,0.796875,0.510638
500,0.171875,0.446809
700,0.027344,0.021277


- 주소를 변경한 경우, 보험 사기와 관련 없이 deductible이 높아진다.
- 특히, 보험 사기인 경우 그 비율이 더 많이 높아진다

In [46]:
# 주소이전하고 보험사기인 경우 Deductible을 더 많이 냄
all_p.Deductible.mean()

407.6196898179366

In [47]:
# change가 왜 더 높은지.. 비즈니스적으로 설명할 수 있어야함
all_p.groupby(['AddressChange_Claim_3']).agg({"Deductible":'mean'})

,Deductible
AddressChange_Claim_3,
change,428.712871
no_change,406.078148


In [48]:
all_p.groupby(['AddressChange_Claim_3', 'FraudFound_P']).agg({"Deductible":'mean'})

Deductible
AddressChange_Claim_3 FraudFound_P            
change                0             425.000000
                      1             448.936170
no_change             0             406.014435
                      1             406.666667

- 주소를 변경한 경우 평균 자기부담금이 406에서 428로 상승함
    - 보험 사기인 경우 자기부담금의 평균이 448까지 올라감

In [49]:
tmp = collision.loc[collision['AddressChange_Claim_3']=='no_change']

statistical_tests_step1(tmp, 'FraudFound_P', ['Deductible'])

pd.crosstab(tmp['Deductible'], tmp['FraudFound_P'], normalize='columns')

FraudFound_P,0,1
Deductible,,
400,0.981966,0.979275
500,0.000194,0.002591
700,0.017840,0.018135


In [50]:
tmp = collision.loc[collision['AddressChange_Claim_3']=='change']

statistical_tests_step1(tmp, 'FraudFound_P', ['Deductible'])

plot_count_plot(tmp, 'Deductible', 'FraudFound_P')

Relevant : Deductible
FraudFound_P


In [51]:
collision.Deductible.mean()

406.77624958067764

In [52]:
pd.crosstab(tmp['Deductible'], tmp['FraudFound_P'], normalize='columns')

FraudFound_P,0,1
Deductible,,
300,0.005405,0.020408
400,0.794595,0.489796
500,0.186486,0.489796
700,0.013514,0.000000


In [53]:
collision.groupby(['AddressChange_Claim_3']).agg({"Deductible":'mean'})

,Deductible
AddressChange_Claim_3,
change,425.059666
no_change,405.394191


In [54]:
collision.groupby(['AddressChange_Claim_3', 'FraudFound_P']).agg({"Deductible":'mean'})

Deductible
AddressChange_Claim_3 FraudFound_P            
change                0             422.162162
                      1             446.938776
no_change             0             405.371340
                      1             405.699482

collision도 동일한 패턴을 보임

- all perils와 collision 모두, 주소를 이전한 경우에 대해 자기부담금의 금액이 보험 사기 여부 판단에 도움이 된다.
    - (비즈니스적 해석) 자기부담금이 높을 수록 더 많은 보험금을 제공해주기 때문인가?

> # 마무리

- (기준에 대한 구체화) 주소를 언제 이전? 최근? 옛날에? 최근이나 옛날의 기준은?
    - 주소를 이전한 경우 보험사기의 비율이 높다
        - 주소를 이전한 경우 15%가 보험 사기, 주소를 이전하지 않은 경우는 9%가 보험 사기.
- (다른 변수와의 관계 구체화) 주소를 이사로 인해 이전한 경우와 어떻게 구분하지?
    - 단순한 이사로 인해 변경되었는지 여부는 판단할 수 없음
    - 대신, 주소 이전을 하고 보험 사기 대상인 피보험자들의 특징을 파악할 수는 있음
        - 자기 부담금이 높은 경향이 있으며, 여러대의 차량 보다는 1대의 차량을 갖고 있는 피보험자가 상대적으로 많다
- (비즈니스 근거)
    - 주소를 도용하여 주소 변경, 은행 명세서나 운전면허 갱신서 등 민감한 정보를 열람하여 범죄에 활용
- (결론 및 활용) 비용/자원 여부와 비즈니스 타당성 여부를 함께 고려
    - 보험 사기의 특징을 담을 수 있는 data point 필요 (주소 이전을 통한 보험 사기)
    - 주소 이전을 통한 보험사기와 높은 자기 부담금의 상관성 탐색 필요
    - 주소 이전을 통한 보험사기와 차량의 대수의 상관성 탐색 필요

---
## #추가

보험 사기인 경우 1대인 경우가 많은데, 만약 자동차 대수 == 부의 상징 이라면, 주소 이전을 했지만 부자인 경우와, 부자가 아닌 경우에 따라서 나눌 수 있지 않을까?

In [ ]:
numcars_mapping = {
    '1 vehicle' : 1,
    '2 vehicles' : 2,
    '3 to 4' : 3.5,
    '5 to 8' : 6.5,
    'more than 8' : 8
}

collision['NumberOfCars_num'] = collision['NumberOfCars'].map(numcars_mapping)
all_p['NumberOfCars_num'] = all_p['NumberOfCars'].map(numcars_mapping)

In [ ]:
vehprice_mapping = {
    'less than 20000': 15000,
    '20000 to 29000': 25000,
    '30000 to 39000': 35000,
    '40000 to 59000': 50000,
    '60000 to 69000': 65000,
    'more than 69000': 70000,
}

collision['VehiclePrice_num'] = collision['VehiclePrice'].map(vehprice_mapping)
all_p['VehiclePrice_num'] = all_p['VehiclePrice'].map(vehprice_mapping)

In [ ]:
collision['wealth'] = collision['NumberOfCars_num'] * collision['VehiclePrice_num']
all_p['wealth'] = all_p['NumberOfCars_num'] * all_p['VehiclePrice_num']

In [ ]:
collision['wealth'].hist();

In [ ]:
all_p['wealth'].hist();

In [ ]:
collision_change = collision.loc[collision['AddressChange_Claim_3']=='change']
all_p_change = all_p.loc[all_p['AddressChange_Claim_3']=='change']

In [ ]:
o, c = statistical_tests_step1(collision_change, 'FraudFound_P', ['NumberOfCars_num', 'VehiclePrice_num', 'wealth'])
o, c = statistical_tests_step1(all_p_change, 'FraudFound_P', ['NumberOfCars_num', 'VehiclePrice_num', 'wealth'])